In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib



In [18]:
df = pd.read_csv('Dataset/Training.csv')
df


,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [19]:
df.shape
column = list(df.columns)


In [20]:
print(column)

['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs', 'fast_heart_rate', 'pain_during_bow

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

# Load your dataset
df = pd.read_csv('Dataset/training.csv')  # Replace with your dataset file path

# 1. Feature and label preparation
X = df.drop('prognosis', axis=1).values  # Ensures X is a NumPy array
y = df['prognosis']

# 2. Encode labels
le = LabelEncoder()
Y = le.fit_transform(y)  # Encoded labels

# Save the LabelEncoder for later use
joblib.dump(le, 'label_encoder.pkl')


def add_noise(X, noise_level=0.05):
    X_noisy = X.copy()
    mask = np.random.rand(*X.shape) < noise_level
    X_noisy[mask] = 1 - X_noisy[mask]  # Flip 0<->1
    return X_noisy

X_noisy = add_noise(X, noise_level=0.05)


# 3. Split unscaled data (for all models except MultinomialNB)
X_train, X_test, y_train, y_test = train_test_split(X_noisy, Y, test_size=0.2, random_state=42)

# 4. Scale data for MultinomialNB (which requires non-negative inputs)
scaler = MinMaxScaler()
X_scaled_noisy = scaler.fit_transform(X_noisy).astype(np.float64)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    X_scaled_noisy, Y, test_size=0.2, random_state=42
)

# 5. Define models
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB()
}



# 6. Training and evaluation
best_model = None
best_accuracy = 0

for model_name, model in models.items():
    print(f"Training and testing {model_name}...")
    
    if model_name == 'MultinomialNB':
        model.fit(X_train_scaled, y_train_scaled)
        predictions = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, predictions)
    print(f"{model_name} Confusion Matrix:")
    print(np.array2string(cm, separator=', '))

    # Save the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

    print("\n" + "="*40 + "\n")


Training and testing SVC...
SVC Accuracy: 0.9878
SVC Confusion Matrix:
[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 30,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 25,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 32]]


Training and testing RandomForest...
RandomForest Accuracy: 0.9939
RandomForest Confusion Matrix:
[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 29,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 25,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 34]]


Training and testing GradientBoosting...
GradientBoosting Accuracy: 0.9827
GradientBoosting Confusion Matrix:
[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 29,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 1,  0,  0, ..., 24,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 32]]


Training and testing KNeighbors...
KNeighbors Accuracy: 0.9898
KNeighbors Confusion Matrix:
[[18,  0,  0, ...,  0,  0,  0

In [22]:
print("Training accuracy:", model.score(X_train, y_train))
print("Testing accuracy:", model.score(X_test, y_test))


Training accuracy: 0.9928861788617886
Testing accuracy: 0.9817073170731707


In [23]:
print("Unique rows in X:", len(np.unique(X, axis=0)))
print("Total rows:", len(X))


Unique rows in X: 304
Total rows: 4920


In [24]:
print(df['prognosis'].nunique())  # How many different diseases
print(df.groupby('prognosis').size().sort_values(ascending=False))

41
prognosis
(vertigo) Paroymsal  Positional Vertigo    120
AIDS                                       120
Acne                                       120
Alcoholic hepatitis                        120
Allergy                                    120
Arthritis                                  120
Bronchial Asthma                           120
Cervical spondylosis                       120
Chicken pox                                120
Chronic cholestasis                        120
Common Cold                                120
Dengue                                     120
Diabetes                                   120
Dimorphic hemmorhoids(piles)               120
Drug Reaction                              120
Fungal infection                           120
GERD                                       120
Gastroenteritis                            120
Heart attack                               120
Hepatitis B                                120
Hepatitis C                                120


In [34]:
# selecting svc
svc = SVC(kernel='linear', probability = True)
svc.fit(X_train,y_train)
ypred = svc.predict(X_test)
accuracy_score(y_test,ypred)

0.9878048780487805

In [35]:
# save svc
import pickle
pickle.dump(svc,open('svc.pkl','wb'))

In [36]:
svc = pickle.load(open('svc.pkl','rb'))

In [30]:
print("Predicted Disease:", le.inverse_transform(svc.predict(X_test[3].reshape(1, -1)))[0])
print("Actual Disease:", le.inverse_transform([y_test[3]])[0])


Predicted Disease: AIDS
Actual Disease: AIDS


In [37]:
print("Predicted Disease:", le.inverse_transform(svc.predict(X_test[115].reshape(1, -1)))[0])
print("Actual Disease:", le.inverse_transform([y_test[115]])[0])


Predicted Disease: Drug Reaction
Actual Disease: Drug Reaction


In [38]:
sym_des = pd.read_csv("Dataset/symtoms_df.csv")
precautions = pd.read_csv("Dataset/precautions_df.csv")
workout = pd.read_csv("Dataset/workout_df.csv")
description = pd.read_csv("Dataset/description.csv")
medications = pd.read_csv('Dataset/medications.csv')
diets = pd.read_csv("Dataset/diets.csv")

In [41]:
def helper(dis):
    desc = description[description['Disease'] == dis]['Description']
    desc = " ".join([w for w in desc])

    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications[medications['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets[diets['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout[workout['disease'] == dis]['workout']

    return desc, pre, med, die, wrkout

symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}

def get_symptom_contributions(patient_symptoms, disease_idx):
    """
    Calculate how much each symptom contributes to the disease prediction
    """
    # Create base input vector with selected symptoms
    base_vector = np.zeros(len(symptoms_dict))
    valid_symptoms = []
    
    for symptom in patient_symptoms:
        if symptom in symptoms_dict:
            base_vector[symptoms_dict[symptom]] = 1
            valid_symptoms.append(symptom)
    
    if not valid_symptoms:
        return []
    
    # Get base probability for the disease
    base_prob = svc.predict_proba([base_vector])[0][disease_idx]
    
    contributions = []
    
    # Calculate contribution of each symptom
    for symptom in valid_symptoms:
        # Create vector without this symptom
        test_vector = base_vector.copy()
        test_vector[symptoms_dict[symptom]] = 0
        
        # Get probability without this symptom
        prob_without = svc.predict_proba([test_vector])[0][disease_idx]
        
        # Calculate contribution (difference in probability)
        contribution = base_prob - prob_without
        contribution_percentage = (contribution / base_prob) * 100 if base_prob > 0 else 0
        
        contributions.append({
            'symptom': symptom,
            'contribution': round(contribution_percentage, 2),
            'probability_impact': round(contribution * 100, 2)
        })
    
    # Sort by contribution (highest first)
    contributions.sort(key=lambda x: x['contribution'], reverse=True)
    
    return contributions

# Enhanced prediction function with symptom contributions
def get_predicted_value_with_contributions(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    valid_symptoms = []
    
    for item in patient_symptoms:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
            valid_symptoms.append(item)
    
    # Get prediction probabilities
    probabilities = svc.predict_proba([input_vector])[0]
    
    # Get top 3 predictions
    top_indices = np.argsort(probabilities)[-3:][::-1]
    
    results = []
    for idx in top_indices:
        disease = diseases_list[idx]
        
        # Model confidence
        model_confidence = probabilities[idx] * 100
        
        # Symptom match percentage
        symptom_match = (len(valid_symptoms) / len(patient_symptoms)) * 100 if patient_symptoms else 0
        
        # Get symptom contributions for this disease
        symptom_contributions = get_symptom_contributions(patient_symptoms, idx)
        
        results.append({
            'disease': disease,
            'confidence': round(model_confidence, 2),
            'symptom_match': round(symptom_match, 2),
            'valid_symptoms': len(valid_symptoms),
            'total_symptoms': len(patient_symptoms),
            'symptom_contributions': symptom_contributions
        })
    
    return results


In [42]:
test_symptoms = ['high_fever', 'headache', 'nausea', 'vomiting']
results = get_predicted_value_with_contributions(test_symptoms)

# Display results
for i, result in enumerate(results, 1):
    print(f"\n=== Prediction {i}: {result['disease']} ===")
    print(f"Confidence: {result['confidence']}%")
    print(f"Symptom Match: {result['symptom_match']}%")
    print(f"\nSymptom Contributions:")
    print("-" * 50)
    for contrib in result['symptom_contributions']:
        print(f"{contrib['symptom']:<20}: {contrib['contribution']:>6.2f}% contribution")


=== Prediction 1: (vertigo) Paroymsal  Positional Vertigo ===
Confidence: 15.52%
Symptom Match: 100.0%

Symptom Contributions:
--------------------------------------------------
nausea              :  70.56% contribution
headache            :  56.34% contribution
vomiting            :  25.49% contribution
high_fever          : -35.46% contribution

=== Prediction 2: Paralysis (brain hemorrhage) ===
Confidence: 14.38%
Symptom Match: 100.0%

Symptom Contributions:
--------------------------------------------------
headache            :  59.00% contribution
vomiting            :  49.45% contribution
high_fever          : -43.46% contribution
nausea              : -86.26% contribution

=== Prediction 3: Malaria ===
Confidence: 13.37%
Symptom Match: 100.0%

Symptom Contributions:
--------------------------------------------------
nausea              :  64.89% contribution
high_fever          :  54.60% contribution
headache            :  52.19% contribution
vomiting            :  47.00% con